In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import folium
import requests
from folium.plugins import HeatMap, HeatMapWithTime

In [2]:
df = pd.read_csv("Dataset/Sample_NaStreet_Removed.csv")
df_street=pd.read_csv("street.csv")

In [10]:
#df.iloc[:,25:30].head()
#df.isnull().sum()
#df.columns
df.describe()

,Unnamed: 0,Unnamed: 0.1,Summons Number,Violation Code,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,...,Date First Observed,Law Section,Unregistered Vehicle?,Vehicle Year,Feet From Curb,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,count,Hour
count,1.077238e+06,1.077238e+06,1.077238e+06,1.077238e+06,1.077238e+06,1.077238e+06,1.077238e+06,1.077238e+06,870786.000000,1.077238e+06,...,1.077238e+06,1.077238e+06,112214.0,1.077238e+06,1.077238e+06,0.0,0.0,0.0,1077238.0,1.077238e+06
mean,5.401314e+05,5.401491e+06,6.819190e+09,3.461030e+01,2.391182e+04,2.034024e+04,2.036258e+04,2.443127e+07,55.718660,4.504021e+01,...,4.530347e+05,5.511100e+02,0.0,1.577745e+03,1.203188e-01,NaN,NaN,NaN,1.0,1.156734e+01
std,3.118598e+05,3.118332e+06,2.318808e+09,1.935773e+01,2.268539e+04,2.195763e+04,2.201386e+04,2.603851e+07,37.948637,4.055940e+01,...,2.988324e+06,2.950041e+02,0.0,8.257774e+02,8.306455e-01,NaN,NaN,NaN,0.0,3.988754e+00
min,0.000000e+00,3.000000e+00,1.005796e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,...,0.000000e+00,4.080000e+02,0.0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,1.0,0.000000e+00
25%,2.700552e+05,2.698808e+06,5.092154e+09,2.000000e+01,5.580000e+03,0.000000e+00,0.000000e+00,2.016072e+07,19.000000,7.000000e+00,...,0.000000e+00,4.080000e+02,0.0,1.999000e+03,0.000000e+00,NaN,NaN,NaN,1.0,9.000000e+00
50%,5.401135e+05,5.400448e+06,8.107033e+09,3.600000e+01,1.813000e+04,1.383000e+04,1.384000e+04,2.017083e+07,50.000000,3.300000e+01,...,0.000000e+00,4.080000e+02,0.0,2.008000e+03,0.000000e+00,NaN,NaN,NaN,1.0,1.100000e+01
75%,8.101878e+05,8.102525e+06,8.485913e+09,4.000000e+01,3.549000e+04,3.391000e+04,3.399000e+04,2.018072e+07,90.000000,7.800000e+01,...,0.000000e+00,4.080000e+02,0.0,2.014000e+03,0.000000e+00,NaN,NaN,NaN,1.0,1.400000e+01
max,1.080302e+06,1.080303e+07,8.585600e+09,9.900000e+01,9.802000e+04,9.826000e+04,9.828000e+04,8.888889e+07,878.000000,8.780000e+02,...,2.022061e+07,1.180000e+03,0.0,2.069000e+03,1.600000e+01,NaN,NaN,NaN,1.0,2.300000e+01


In [4]:
print(len(df))
invalid = df.loc[(df['Latitude']=='n') & (df['Longitude']=='o')]
print(len(invalid))
df = df[(df['Latitude']!='n') & (df['Longitude']!='o')]
df = df[pd.notnull(df.Hour)]
print(len(df))

1079918
2665
1077238


In [5]:
df_hour=[]
for hour in df.Hour.sort_values().unique():
    df_hour.append(df.loc[df.Hour == hour, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [6]:
len(df_hour)

24

In [7]:
df.Hour.sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [8]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [9]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map